# Student Retention Predictor
> Predicting college dropout risk using U.S. Department of Education data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

In [3]:
df = pd.read_csv('../data/college_scorecard_apr2025.csv', low_memory=False)